* Make Changes to the Application
* Setup Supervisor on GCP VM
* Create env file on VM with Database Credentials
* Shell Script to Build and Run Application
* Update GitHub Action Workflow file
* Define Supervisor Configuration File
* Merge Changes to Main Branch
* Validate CI/CD Pipeline
* Cleanup CICD Demo Application
* Exercise and Solution

* Make Changes to the Application

Here are the steps to create a branch.
* Checkout a new branch
* Update the requirements file

```text
Flask==2.2.3
Flask-SQLAlchemy==3.0.3
psycopg2-binary==2.9.6
gunicorn==20.1.0
python-dotenv==1.0.0
```

* Make changes to the application and test it locally

```python
import os
from dotenv import load_dotenv
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

load_dotenv('.env')
db = SQLAlchemy()
app = Flask(__name__)
host = os.environ.get('CICD_DB_HOST')
port = os.environ.get('CICD_DB_PORT')
db_name = os.environ.get('CICD_DB_NAME')
user = os.environ.get('CICD_DB_USER')
password = os.environ.get('CICD_DB_PASS')
app.config["SQLALCHEMY_DATABASE_URI"] = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'
db.init_app(app)


class User(db.Model):
    __tablename__ = 'users' # Model and table name are different
    id = db.Column(db.Integer, primary_key=True, )
    first_name = db.Column(db.String)
    last_name = db.Column(db.String)
    username = db.Column(db.String, unique=True, nullable=False)
    email = db.Column(db.String)

    def __repr__(self):
        return f'<user_id={self.id};user_first_name={self.first_name};user_last_name={self.last_name}>'


@app.route('/')
def hello_world():
    user = db.get_or_404(User, 1)
    return f"Hello, World from {user.first_name}"
```

* Add, Commit and Push the changes in the new branch to Git repository.

* Setup Supervisor on GCP VM

Here are the commands to run supervisor.

```shell
sudo apt update
sudo apt install -y supervisor
```

* Create env file on VM with Database Credentials

Add `.env` under Project Directory on GCP VM with Database details.

```shell
CICD_DB_HOST=<POSTGRES_DB_HOST>
CICD_DB_PORT=<POSTGRES_DB_PORT>
CICD_DB_NAME=<POSTGRES_DB_NAME>
CICD_DB_USER=<POSTGRES_DB_USER>
CICD_DB_PASS=<POSTGRES_DB_PASS>
```

* Shell Script to Build and Run the Application

Add setup.sh in the base folder for the project.

```shell
python3 -m venv /home/itversity/cicd-demo/cd-venv
source /home/itversity/cicd-demo/cd-venv/bin/activate
pip install -r /home/itversity/cicd-demo/requirements.txt
sudo supervisorctl restart cicd-demo
```

* Update GitHub Action Workflow file

```yml
name: CICD Demo

on:
  push:
    branches: [ "main" ]
  pull_request:
    branches: [ "main" ]

permissions:
  contents: read

jobs:
  build:

    runs-on: ubuntu-latest

    steps:
    - uses: actions/checkout@v3
    - name: Set Environment Variables
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        mkdir -p /tmp/keys && chmod 700 /tmp/keys
        echo "$APP_SSH_SECRET_KEY" > /tmp/keys/private_key && chmod 600 /tmp/keys/private_key
    - name: Deploy Application
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        ssh -o StrictHostKeyChecking=no -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} '
          mkdir -p ~/cicd-demo
        '
        scp -i /tmp/keys/private_key * ${APP_SSH_USER}@${APP_HOST}:~/cicd-demo
    - name: Build and Run Application
      env:
        APP_HOST: ${{ secrets.APP_HOST }}
        APP_SSH_USER: ${{ secrets.APP_SSH_USER }}
        APP_SSH_SECRET_KEY: ${{ secrets.APP_SSH_SECRET_KEY }}
      run: |
        ssh -i /tmp/keys/private_key ${APP_SSH_USER}@${APP_HOST} "/bin/bash -c 'source ~/cicd-demo/setup.sh'"
```

* Define Supervisor Configuration File

Add below content to `/etc/supervisor/conf.d/cicd.conf`

```text
[program:cicd-demo]
directory=/home/itversity/cicd-demo
command=/home/itversity/cicd-demo/cd-venv/bin/gunicorn app:app -b localhost:5000
autostart=true
autorestart=true
stderr_logfile=/var/log/cicd-demo/cicd-demo.err.log
stdout_logfile=/var/log/cicd-demo/cicd-demo.out.log
```

* Merge Changes to Main Branch and Validate

Make sure to place PR and complete the process to merge changes to main branch. Go to Action and ensure if every thing is running successfully or not.

Also access the application using browser and see if it is working fine or not.

You can also login into the GCP VM and run `sudo supervisorctl status cicd-demo` to confirm if cicd-demo is running or not using gunicorn.

* Cleanup CICD Demo Application

Here are the instructions to cleanup CICD Demo Application.
1. Stop supervisor
2. Delete cicd supervisor configuration file
3. Start supervisor

* Exercise - CI/CD using GitHub Actions for sales-app

1. Review existing GitHub Action Workflow file for sales-app
2. Add Secrets to deploy the application via GitHub Actions (if missing)

|Secret Name|Description|
|---|---|
|APP_HOST|IP Address or DNS Alias of VM|
|APP_SSH_USER|Username to connect to VM|
|APP_SSH_SECRET_KEY|Private key for password less login|

3. Make sure GitHub Action Workflow file is `sales-app.yml`. It should have steps to setup password less login and deploy the application.
4. Update `nginx.conf` with correct details (if applicable) to proxy to sales-app.
5. Create supervisor configuration file by name `sales-app.conf` in GCP VM. Validate all the paths for the folders specified in configuration file. Make sure to restart supervisor.
6. Define .env file for Postgres credentials. Make required changes to the application to read environment variables from .env. Also add .env to .gitignore so that the file is not pushed to GitHub. Each environment should have its own version of .env.
7. Add required shell script to the base folder of the repository to take care of build and run. Make sure to add step to yml file to trigger shell script to take care of build and run.
8. Merge the changes to main branch and then validate whether GitHub Action is run successfully or not.

* Solution - CI/CD using GitHub Actions for sales-app

1. Review existing GitHub Action Workflow file for sales-app
2. Add Secrets to deploy the application via GitHub Actions (if missing)

|Secret Name|Description|
|---|---|
|APP_HOST|IP Address or DNS Alias of VM|
|APP_SSH_USER|Username to connect to VM|
|APP_SSH_SECRET_KEY|Private key for password less login|

3. Make sure GitHub Action Workflow file is `sales-app.yml`. It should have steps to setup password less login and deploy the application.
4. Update `nginx.conf` with correct details (if applicable) to proxy to sales-app.
5. Create supervisor configuration file by name `sales-app.conf` in GCP VM. Validate all the paths for the folders specified in configuration file. Make sure to restart supervisor.

```text
[program:sales-app]
directory=/home/itversity/sales-app
command=/home/itversity/sales-app/cd-venv/bin/gunicorn app:app -b localhost:5000
autostart=true
autorestart=true
stderr_logfile=/var/log/sales-app/sales-app.err.log
stdout_logfile=/var/log/sales-app/sales-app.out.log
```

Restart supervisor using `sudo systemctl restart supervisor`. Make sure to validate by saying `sudo supervisorctl status`.

6. Define .env file for Postgres credentials. Make required changes to the application to read environment variables from .env. Also add .env to .gitignore so that the file is not pushed to GitHub. Each environment should have its own version of .env.

```shell
SALES_DB_HOST=<POSTGRES_DB_HOST>
SALES_DB_PORT=<POSTGRES_DB_PORT>
SALES_DB_NAME=<POSTGRES_DB_NAME>
SALES_DB_USER=<POSTGRES_DB_USER>
SALES_DB_PASS=<POSTGRES_DB_PASS>
```

Update `requirements.txt` with `python-dotenv`.

```text
Flask==2.3.2
Flask-SQLAlchemy==3.0.3
Flask-Bootstrap==3.3.7.1
psycopg2-binary==2.9.6
gunicorn==20.1.0
python-dotenv==1.0.0
```

Update `app.py` with logic to read environment variables from `.env`.

```python
import os
from dotenv import load_dotenv
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy
from flask_bootstrap import Bootstrap

load_dotenv('.env')
db = SQLAlchemy()
app = Flask(__name__)
Bootstrap(app)
host = os.environ.get('SALES_DB_HOST')
port = os.environ.get('SALES_DB_PORT')
db_name = os.environ.get('SALES_DB_NAME')
user = os.environ.get('SALES_DB_USER')
password = os.environ.get('SALES_DB_PASS')
app.config['SQLALCHEMY_DATABASE_URI'] = f'postgresql://{user}:{password}@{host}:{port}/{db_name}'
db.init_app(app)
from models.user import User
from models.course import Course


@app.route('/')
def hello_world():
    # rec = db.get_or_404(User, 1)
    rec = User.query.get_or_404(1)
    return render_template('index.html', user=rec)


@app.route('/users')
def users():
    user_recs = db.session.query(User).all()
    users = list(map(lambda rec: rec.__dict__, user_recs))
    return render_template('users.html', users=users)


@app.route('/courses')
def courses():
    course_recs = db.session.query(Course).all()
    courses = list(map(lambda rec: rec.__dict__, course_recs))
    return render_template('courses.html', courses=courses)
```

7. Add required shell script to the base folder of the repository to take care of build and run. Make sure to add step to yml file to trigger shell script to take care of build and run.

```shell
python3 -m venv /home/itversity/sales-app/sa-venv
source /home/itversity/sales-app/sa-venv/bin/activate
pip install -r /home/itversity/sales-app/requirements.txt
sudo supervisorctl restart sales-app
```

8. Merge the changes to main branch and then validate whether GitHub Action is run successfully or not.